In [14]:
import pymongo
import secrets
import streamlit as st
import pandas as pd

def connect_to_mongodb():
    secrets = st.secrets["mongo"]
    conn_str = secrets["conn_str"]
    client = pymongo.MongoClient(conn_str)
    db_name = client.ClimbingGradeFeedback
    collection = db_name.ClimbingFeedbackStreamlit

    return collection, client

# Test the connection
try:
    collection, client = connect_to_mongodb()
    # The ismaster command is cheap and does not require auth.
    client.admin.command('ismaster')
    print("MongoDB connection successful")
except Exception as e:
    print("MongoDB connection unsuccessful")
    print(e)

MongoDB connection successful


In [15]:
# Run the function to connect to MongoDB and get the cursor
collection, client = connect_to_mongodb()

# Query the MongoDB collection and get a cursor
cursor = collection.find()

# Create a copy of the cursor for later use
cursor_copy = cursor.clone()

In [16]:
# Convert the original cursor (cursor_copy) to a list and then create a DataFrame
df = pd.DataFrame(list(cursor_copy))

In [17]:
# print the spread of values in the actual_sport_grade column and actual_boulder_grade column
# this will help us to see if the data is clean, and if there are any values that need to be removed or changed
print(df.actual_sport_grade.value_counts())


actual_sport_grade
6b+                                    20
                                       16
6b                                      8
6c                                      7
6a                                      6
8a                                      5
7a                                      5
7c                                      5
6c+                                     4
7b                                      4
7b+                                     4
6b                                      3
7a+                                     3
V4                                      3
7c+                                     3
8a+                                     2
5.12d                                   2
8c+                                     2
6b+                                     2
5.11                                    2
5.13                                    2
Na                                      2
8b                                      1
7A+            

In [18]:
print(df.actual_bouldering_grade.value_counts())

actual_bouldering_grade
V6         17
V5         13
v8         12
7b         10
V4         10
V11         9
V8          9
V9          6
V7          4
7a          4
7A          3
v6          3
v7          3
7B          2
V10         2
6b          2
v5          2
6c          2
6a          2
 V6         1
4           1
V3          1
v3          1
v13         1
7c+         1
V12         1
7B+         1
{{7*7}}     1
Name: count, dtype: int64


In [25]:
# List of grades to exclude
excluded_grades = ['4c', '5a', '5b', '5c', '6a', '6a+', '6b', '6b+', '6c', '6c+', '7a', '7a+', '7b', '7b+', '7c', '7c+', '8a', '8a+', '8b', '8b+', '8c', '8c+', '9a', '9a+', '0']

# Filter the DataFrame to get rows with max_sport_grade not in the excluded list
filtered_df = df[~df['actual_sport_grade'].isin(excluded_grades)]

# Select specific columns from the filtered DataFrame
selected_columns = ['predicted_sport_grade', 'actual_sport_grade']
filtered_df = filtered_df[selected_columns]

# Display the resulting DataFrame
print(filtered_df)

    predicted_sport_grade                   actual_sport_grade
2                     7a+                                     
7                     7a+                                5.12c
8                     7a+                                     
11                     7a                                   28
14                    6a+                                     
17                     6b                                  6b 
18                     6b                                  6b 
19                     6b                                  6b 
22                     7a                                     
25                    6c+  N/A (like 11b, i don't sport climb)
26                     7a                                 12.b
28                     7a                                     
29                     7a                                     
30                     7a                                   Na
31                     7a                              

In [29]:
# based on the above results, clean the data as needed
df.fillna(0, inplace=True)
df.replace('', 0, inplace=True)
df.replace('Na', 0, inplace=True)
df.replace('N/A (like 11b, i don\'t sport climb)', 0, inplace=True)
df.replace('Unknown', 0, inplace=True)
df.replace('V4', 0, inplace=True)
df.replace('28', 0, inplace=True)
df.replace('12.b', 0, inplace=True)
df.replace('5.12', '7a', inplace=True)
df.replace('5.12a', '7a+', inplace=True)
df.replace('5.12b', '7b', inplace=True)
df.replace('5.12c', '7b+', inplace=True)
df.replace('5.12d', '7c', inplace=True)
df.replace('5.13', '7c', inplace=True)
df.replace('5.13a', '7c+', inplace=True)
df.replace('5.11', '6b+', inplace=True)
df.replace('5.11a', '6c', inplace=True)
df.replace('5.11b', '6c', inplace=True)
df.replace('5.11c', '6c+', inplace=True)
df.replace('5.11d', '7a', inplace=True)
df.replace('6B+', '6b+', inplace=True)
df.replace('6b+ ', '6b+', inplace=True)
df.replace('6b ', '6b+', inplace=True)
df.replace('6C', '6c', inplace=True)
df.replace('7A+', '7a+', inplace=True)
df.replace('7B+', '7b+', inplace=True)

In [30]:
# print the spread of values in the actual_sport_grade column and actual_boulder_grade column
# this will help us to see if the data is clean, and if there are any values that need to be removed or changed
print(df.actual_sport_grade.value_counts())

actual_sport_grade
6b+    28
0      25
6c      9
7c      9
6b      8
6a      6
7b+     6
7a      5
6c+     5
7b      5
8a      5
7a+     4
7c+     3
8c+     2
8a+     2
8b      1
5c      1
Name: count, dtype: int64


In [36]:
# Define the V grade list
v_grade_list = ["<V3", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16"]

# Filter the DataFrame to get rows with max_bouldering_grade in the V grade list
filtered_df = df[~df['actual_bouldering_grade'].isin(v_grade_list)]


# Select specific columns from the filtered DataFrame
selected_columns = ['predicted_bouldering_grade', 'actual_bouldering_grade']
filtered_df = filtered_df[selected_columns]

# Display the resulting DataFrame
print(filtered_df)

    predicted_bouldering_grade actual_bouldering_grade
2                          V10                      v8
3                            0                       0
5                           V5                      7a
6                            0                       0
8                           V5                     7b+
..                         ...                     ...
115                         V6                 {{7*7}}
118                         V7                      v7
119                         V5                      v7
122                         V6                      7a
123                         V6                      7a

[62 rows x 2 columns]


In [68]:
print(df.columns)
print(len(df))

Index(['_id', 'strength_to_weight_pullup', 'strength_to_weight_maxhang',
       'strength_to_weight_weightpull', 'continuous', 'repeaters1', 'exp',
       'trainexp', 'days', 'predicted_bouldering_grade',
       'actual_bouldering_grade', 'predicted_sport_grade',
       'actual_sport_grade', 'height', 'weight'],
      dtype='object')
51


In [69]:
# remove the '_id' and predicted fields field and rename columns
new_data_mongodb = df.drop(['_id','predicted_bouldering_grade','predicted_sport_grade','height','weight'], axis=1, inplace=True)
new_data_mongodb = df.rename(columns={
    'actual_bouldering_grade': 'max_boulder_grade',
    'actual_sport_grade': 'max_sport_grade'
})

In [ ]:
# save the new data to include in the models
new_data_mongodb.to_csv('new_data_mongodb.csv', index=False)